In [4]:
from utils import list_images, read_image, extract_pixel_size, correct_pixel_size, make_isotropic, remove_labels_touching_edge, simulate_cytoplasm
from pathlib import Path
import os
from tqdm import tqdm
import pyclesperanto_prototype as cle
from skimage.measure import regionprops_table
import pandas as pd

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (2 refs)>

In [5]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data")

# List of subfolder names
subfolder_names = [folder.name for folder in directory_path.iterdir() if folder.is_dir()]

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr),(..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("c3neo", 1), ("myc-tag", 2)]

markers = [("c3neo", 1), ("myc-tag", 2)]
nuclei_channel = 0

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = 4 # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

In [6]:
# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# Create a 'results' folder in the root directory
results_folder = Path("results") / experiment_id 

try:
    os.makedirs(results_folder)
    print(f"'{results_folder}' folder created successfully.")
except FileExistsError:
    print(f"'{results_folder}' folder already exists.")

for folder_name in tqdm(subfolder_names):

    print(f'\nAnalyzing condition:{folder_name}')

    subfolder_path = directory_path / folder_name

    # Iterate through the .czi and .nd2 files in the subfolder directory
    images = list_images(subfolder_path)

    # Empty list to hold the per image dataframes
    df_imgs_list = []

    for image in tqdm(images):

        img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

        # Obtain xyz pixel (voxel) size from image metadata (nd2 files)
        voxel_size = extract_pixel_size(images[0])

        # Correct pixel size if slicing is applied
        pixel_size_x, pixel_size_y, voxel_size_z = correct_pixel_size(voxel_size, slicing_factor_xy, slicing_factor_z)

        nuclei_resampled = make_isotropic(img[nuclei_channel], pixel_size_x, pixel_size_y, voxel_size_z)

        # Remove background with a top_hat_filter
        #background_subtracted = cle.top_hat_box(nuclei_resampled, radius_x=5, radius_y=5, radius_z=5)

        # Apply gaussian blur to prevent the formation of holes upon labeling
        post_gaussian = cle.gaussian_blur(nuclei_resampled, sigma_x=2, sigma_y=2, sigma_z=2)

        # Voronoi-Otsu labeling
        segmented = cle.voronoi_otsu_labeling(post_gaussian, spot_sigma=10, outline_sigma=1)

        # Close holes in labels to avoid false emtpy volumes within the nuclei
        segmented = cle.closing_labels(segmented, radius=5)

        # Remove labels touching image xy borders
        nuclei_labels = remove_labels_touching_edge(segmented.get())

        # Simulate cytoplasm
        cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=10)

        # Create an  empty list to hold the per channel info
        props_list = []

        # Create a dictionary containing all image descriptors
        descriptor_dict = {"filename": filename, "condition": folder_name}

        # Loop through each channel, resample it and extract the average intensity within nuclei and cytoplasm
        for channel_name, ch_nr in markers:
            print(f"Analyzing channel: {channel_name}")

            # Resample marker and transform into numpy array
            marker_resampled = make_isotropic(img[ch_nr], pixel_size_x, pixel_size_y, voxel_size_z).get()

            # Extract intensity information from each marker channel
            nuclei_props = regionprops_table(label_image=nuclei_labels,
                                    intensity_image=marker_resampled,
                                    properties=["label", "intensity_mean"])
            
            # Extract intensity information from each marker channel
            cyto_props = regionprops_table(label_image=cytoplasm_labels,
                                    intensity_image=marker_resampled,
                                    properties=["label", "intensity_mean"])
            
            # Convert to dataframe
            nuclei_props_df = pd.DataFrame(nuclei_props)
            cyto_props_df = pd.DataFrame(cyto_props)

            # Rename intensity_mean column to indicate the specific image
            nuclei_props_df.rename(columns={"intensity_mean": f"nuclei_{channel_name}_avg_int"}, inplace=True)
            cyto_props_df.rename(columns={"intensity_mean": f"cyto_{channel_name}_avg_int"}, inplace=True)

            # Merge nuclei and cyto on label
            props_df = pd.merge(nuclei_props_df, cyto_props_df, on='label')

            # Append each props_df to props_list
            props_list.append(props_df)

        # Initialize the df with the first df in the list
        img_props_df = props_list[0]
        # Start looping from the second df in the list
        for df in props_list[1:]:
            img_props_df = img_props_df.merge(df, on="label")

        # Add each key-value pair from descriptor_dict to props_df at the specified position
        insertion_position = 0
        for key, value in descriptor_dict.items():
            img_props_df.insert(insertion_position, key, value)
            insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

        # Append each img_props_df into the list containing the per_image info
        df_imgs_list.append(img_props_df)

    # Concatenate all per_img_props into final_df
    final_df = pd.concat(df_imgs_list, ignore_index=True)

    # Save the df containing per_label results into a CSV file
    final_df.to_csv(results_folder / f'{folder_name}_per_label_avg_int.csv')

'results\raw_data' folder created successfully.


  0%|          | 0/3 [00:00<?, ?it/s]


Analyzing condition:20250516LPSN120_C3neo_myc_c3 deficent




Image analyzed: 01
Original Array shape: (3, 31, 2720, 2720)
Compressed Array shape: (3, 31, 680, 680)


Analyzing channel: c3neo
Analyzing channel: myc-tag


...



Image analyzed: 10
Original Array shape: (3, 31, 2720, 2720)
Compressed Array shape: (3, 31, 680, 680)


Analyzing channel: c3neo
Analyzing channel: myc-tag


 91%|█████████ | 10/11 [00:20<00:02,  2.00s/it]



Image analyzed: 11
Original Array shape: (3, 31, 2720, 2720)
Compressed Array shape: (3, 31, 680, 680)
Analyzing channel: c3neo
Analyzing channel: myc-tag


100%|██████████| 3/3 [01:05<00:00, 21.89s/it]
